# 🛍️ | Cora-For-Zava: 정밀 조정된 모델 배포

환영합니다! 이 노트북에서는 Azure SDK를 사용하여 정밀 조정된 모델을 Azure AI에 배포하는 방법을 안내합니다.

## 🛒 Zava 시나리오

**Cora**는 DIY 애호가를 위한 홈 인테리어 용품을 판매하는 가상의 소매업체 **Zava**의 고객 서비스 챗봇입니다. Zava의 제품과 고객 요구 사항을 더 잘 이해하도록 모델을 정밀 조정한 후, 프로덕션 환경에서 사용하기 위해 Azure AI Foundry에 배포해야 합니다. 이 노트북에서는 완료된 정밀 ​​조정 작업 목록을 확인하고 사용자 지정 모델을 배포하는 과정을 안내합니다.


## 🎯 완성 목표

이 노트북을 마치면 다음을 달성하게 됩니다.
- ✅ 성공적으로 완료된 모든 미세 조정 작업 목록
- ✅ 배포할 미세 조정된 모델 선택
- ✅ Azure AI Foundry에 모델 배포
- ✅ 배포가 프로덕션 환경에서 사용 가능한지 확인

## 💡 학습 내용

- 미세 조정 작업 목록 및 관리 방법
- Azure AI Foundry에 미세 조정된 모델 배포 방법
- 모델 배포 모범 사례
- 배포 상태 확인 방법

미세 조정된 모델을 배포할 준비가 되셨나요? 지금 시작해 봅시다! 🚀

---

### 1. Check Environment Variables

In [1]:
import os

# Check required environment variables
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_SUBSCRIPTION_ID",
    "AZURE_RESOURCE_GROUP",
    "AZURE_AI_FOUNDRY_NAME"
]

missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"❌ Missing environment variables: {', '.join(missing_vars)}")
    print("\nPlease set these variables before continuing.")
else:
    print("✅ All required environment variables are set!")

✅ All required environment variables are set!



### 2. Create Azure OpenAI Client

In [2]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=get_bearer_token_provider(
        DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2025-02-01-preview")
)

print("✅ Azure OpenAI client created successfully!")

✅ Azure OpenAI client created successfully!



### 3. List Available Fine-Tuned Models

In [3]:
# List all fine-tuning jobs
jobs_response = client.fine_tuning.jobs.list()

# Filter for succeeded jobs
succeeded_jobs = [
    job for job in jobs_response.data if job.status == "succeeded"]

if not succeeded_jobs:
    print("❌ No successful fine-tuning jobs found.")
    print("\nPlease complete a fine-tuning job first using 31-basic-finetuning.ipynb")
else:
    print(f"✅ Found {len(succeeded_jobs)} successful fine-tuning job(s):\n")
    for i, job in enumerate(succeeded_jobs, 1):
        print(f"{i}. Job ID: {job.id}")
        print(f"   Model: {job.fine_tuned_model}")
        print(f"   Created: {job.created_at}")
        print(f"   Status: {job.status}")
        print()

✅ Found 2 successful fine-tuning job(s):

1. Job ID: ftjob-c29a542bc37e440db871ecc2b01214df
   Model: gpt-4.1-nano-2025-04-14.ft-c29a542bc37e440db871ecc2b01214df-o3-mini-zava-tone-3a095088
   Created: 1767793440
   Status: succeeded

2. Job ID: ftjob-25dbaf49417c425e80f41e39b73a13d7
   Model: gpt-4o-2024-08-06.ft-25dbaf49417c425e80f41e39b73a13d7
   Created: 1767785734
   Status: succeeded




### 4. Select Model to Deploy

Enter a job ID from the list above  

In [7]:
# Use a job ID (get the model name from the job)
# Replace with your job ID from the list above
# job_id = "input your finetuning job ID here"
job_id = "ftjob-c29a542bc37e440db871ecc2b01214df"

In [8]:
# Retrieve the job to get the fine-tuned model name
job = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model = job.fine_tuned_model

print(f"✅ Selected model: {fine_tuned_model}")
print(f"   From job: {job_id}")

✅ Selected model: gpt-4.1-nano-2025-04-14.ft-c29a542bc37e440db871ecc2b01214df-o3-mini-zava-tone-3a095088
   From job: ftjob-c29a542bc37e440db871ecc2b01214df



### 5. Configure Deployment Settings

In [17]:
import time

# Generate unique deployment name
timestamp = int(time.time())
DEPLOYMENT_NAME = f"60-zava-finetuned-{timestamp}"

# Configure deployment settings
DEPLOYMENT = {
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": fine_tuned_model,
            "version": "1"
        },
    },
    "sku": {
        # Adjust based on your needs (e.g., 250 for DeveloperTier)
        "capacity": 250,
        # "name": "Standard"  # Options: "DeveloperTier", "Standard", "GlobalStandard"
        "name": "GlobalStandard"
    },
}

print(f"📋 Deployment Configuration:")
print(f"   Name: {DEPLOYMENT_NAME}")
print(f"   Model: {fine_tuned_model}")
print(f"   SKU: {DEPLOYMENT['sku']['name']}")
print(f"   Capacity: {DEPLOYMENT['sku']['capacity']}")

📋 Deployment Configuration:
   Name: 60-zava-finetuned-1767823149
   Model: gpt-4.1-nano-2025-04-14.ft-c29a542bc37e440db871ecc2b01214df-o3-mini-zava-tone-3a095088
   SKU: GlobalStandard
   Capacity: 250



### 6. Create Azure Management Client

In [18]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

# Create management client for Azure Cognitive Services
cogsvc_client = CognitiveServicesManagementClient(
    credential=DefaultAzureCredential(),
    subscription_id=os.environ.get("AZURE_SUBSCRIPTION_ID")
)

print("✅ Azure Management client created successfully!")

✅ Azure Management client created successfully!



### 7. Deploy the Fine-Tuned Model

In [19]:
# Submit deployment request
print(f"🚀 Starting deployment of {DEPLOYMENT_NAME}...\n")

deployment = cogsvc_client.deployments.begin_create_or_update(
    resource_group_name=os.environ.get("AZURE_RESOURCE_GROUP"),
    account_name=os.environ.get("AZURE_AI_FOUNDRY_NAME"),
    deployment_name=DEPLOYMENT_NAME,
    deployment=DEPLOYMENT,
)

print(f"✅ Deployment request submitted!")
print(f"\n⏳ Deployment is now provisioning...")
print(f"   This typically takes 3-5 minutes for small models")

🚀 Starting deployment of 60-zava-finetuned-1767823149...

✅ Deployment request submitted!

⏳ Deployment is now provisioning...
   This typically takes 3-5 minutes for small models



### 8. Wait for Deployment to Complete

In [20]:
from IPython.display import clear_output
import time

start_time = time.time()
status = deployment.status()

while status not in ["Succeeded", "Failed"]:
    deployment.wait(5)
    status = deployment.status()
    elapsed_min = int((time.time() - start_time) // 60)
    elapsed_sec = int((time.time() - start_time) % 60)

    clear_output(wait=True)
    print(f"🛳️  Provisioning {DEPLOYMENT_NAME}")
    print(f"📊 Status: {status}")
    print(f"⏱️  Elapsed time: {elapsed_min} minutes {elapsed_sec} seconds")

# Final status
elapsed_min = int((time.time() - start_time) // 60)
elapsed_sec = int((time.time() - start_time) % 60)

if status == "Succeeded":
    print(f"\n🎉 Deployment completed successfully!")
    print(f"⏱️  Total time: {elapsed_min} minutes {elapsed_sec} seconds")
    print(f"\n📝 Deployment Details:")
    print(f"   Name: {DEPLOYMENT_NAME}")
    print(f"   Model: {fine_tuned_model}")
else:
    print(f"\n❌ Deployment failed with status: {status}")

🛳️  Provisioning 60-zava-finetuned-1767823149
📊 Status: Succeeded
⏱️  Elapsed time: 3 minutes 32 seconds

🎉 Deployment completed successfully!
⏱️  Total time: 3 minutes 32 seconds

📝 Deployment Details:
   Name: 60-zava-finetuned-1767823149
   Model: gpt-4.1-nano-2025-04-14.ft-c29a542bc37e440db871ecc2b01214df-o3-mini-zava-tone-3a095088



### 9. Test the Deployed Model

In [21]:
# Test the deployed model with multiple sample prompts
test_prompts = [
    "Can I use extension poles with your roller frames?",
    "Do you have natural bristle brushes?"
]

for i, test_prompt in enumerate(test_prompts, 1):
    print(f"Test {i}/{len(test_prompts)}: Testing deployed model with prompt:")
    print(f"   '{test_prompt}'\n")

    response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,  # Use the deployment name
        messages=[
            {"role": "system", "content": "You are Cora, a polite, factual and helpful assistant for Zava, a DIY hardware store."},
            {"role": "user", "content": test_prompt}
        ],
        max_tokens=150
    )

    print(f"Response from {DEPLOYMENT_NAME}:")
    print(response.choices[0].message.content)
    print("\n" + "="*80 + "\n")

Test 1/2: Testing deployed model with prompt:
   'Can I use extension poles with your roller frames?'

Response from 60-zava-finetuned-1767823149:
At Zava, many of our roller frames are designed to be compatible with extension poles, allowing you to work more comfortably on high or low surfaces. However, it’s important to check the specific model you’re interested in.

Here are some general tips:
- Look for frames that have a threaded end or a universal connector, as these are typically designed to work with extension poles.
- If you’re considering a roller frame for a particular project, please consult the product details or ask a Zava associate for compatibility.
- Many extension poles are adjustable in length and can accommodate a variety of roller frame fittings by using adapters if needed.

If you can specify the particular roller frame you’re considering or have additional questions about your project, I’d be glad to


Test 2/2: Testing deployed model with prompt:
   'Do you have

**Insights**

위 두 예시 모두에서 응답이 "정중하고, 사실적이며, 도움이 되는" Zava 가이드라인을 정확하게 따르고 있음을 확인할 수 있습니다.
- 모든 응답은 이모티콘으로 시작합니다.
- 첫 문장은 항상 사용자에게 감사를 표하는 내용입니다("정중함").
- 다음 문장은 항상 유익한 정보입니다("사실적").
- 마지막 문장은 항상 후속 조치를 제안하는 내용입니다("도움").

또한, 몇 가지 예시를 추가하지 않고도 우리가 원하던 간결한 응답을 얻을 수 있었으며, 이를 통해 프롬프트를 단축하고 토큰 비용과 처리 지연 시간을 모두 절감할 수 있었습니다.

---
### Teardown

Once you are done with this lab, don't forget to tear down the infrastructure. The developer tier model will be torn down automatically (after 24 hours?) but it is better to proactively delete the resource group and release all model quota.